# النواة الدلالية

في هذا المثال البرمجي، ستستخدم إطار العمل [النواة الدلالية](https://aka.ms/ai-agents-beginners/semantic-kernel) للذكاء الاصطناعي لإنشاء وكيل أساسي.

الهدف من هذا المثال هو عرض الخطوات التي سنستخدمها لاحقًا في أمثلة برمجية إضافية عند تنفيذ الأنماط الوكالية المختلفة.


## استيراد حزم بايثون المطلوبة


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## إنشاء العميل

في هذا المثال، سنستخدم [نماذج GitHub](https://aka.ms/ai-agents-beginners/github-models) للوصول إلى نموذج اللغة الكبير (LLM).

تم تعريف `ai_model_id` كـ `gpt-4o-mini`. حاول تغيير النموذج إلى نموذج آخر متاح في سوق نماذج GitHub لرؤية النتائج المختلفة.

لكي نتمكن من استخدام `Azure Inference SDK` الذي يُستخدم لـ `base_url` لنماذج GitHub، سنستخدم الموصل `OpenAIChatCompletion` داخل Semantic Kernel. هناك أيضًا [موصلات أخرى متاحة](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) لاستخدام Semantic Kernel مع مزودي النماذج الآخرين.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## إنشاء الوكيل

فيما يلي نقوم بإنشاء الوكيل المسمى `TravelAgent`.

في هذا المثال، نستخدم تعليمات بسيطة جدًا. يمكنك تغيير هذه التعليمات لمعرفة كيف يستجيب الوكيل بشكل مختلف.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## تشغيل الوكيل

يمكننا الآن تشغيل الوكيل من خلال تعريف سلسلة من النوع `ChatHistoryAgentThread`. يتم توفير أي رسائل نظام مطلوبة إلى الوسيط عبر الوسيطة `messages` في `invoke_stream`.

بعد تعريف هذه الرسائل، نقوم بإنشاء `user_inputs` وهو ما سيرسله المستخدم إلى الوكيل. في هذه الحالة، قمنا بتعيين هذه الرسالة إلى `Plan me a sunny vacation`.

يمكنك تغيير هذه الرسالة لتلاحظ كيف يستجيب الوكيل بشكل مختلف.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**إخلاء المسؤولية**:  
تمت ترجمة هذه الوثيقة باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار الوثيقة الأصلية بلغتها الأصلية المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
